<center>
    <font size="5"> Zaawansowane Metody Uczenia Maszynowego i Głębokiegio<br/>
        <small><em>Studia stacjonarne II stopnia 2024/2025</em><br/>Kierunek: Informatyka<br>Specjalność: Systemy inteligentne i rozszerzona rzeczywistość</small>
    </font>
</center>
<br>



# Laboratorium nr 5: Konwolucyjne sieci neuronowe: Zadanie

## Montowanie Google Drive

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
# uaktualnij poniższą ścieżkę
path_nb = r'/content/drive/My Drive/Colab Notebooks/ZMUMiG_2024_25/'
sys.path.append(path_nb)

## Import bibliotek

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
print('Numpy version:', np.__version__)
print('Tensorflow version:', tf.__version__)

## Zadanie

Stwórz i wytrenuj głęboką konwolucyjną sieć neuronową zdolną poprawnie sklasyfikować zdjęcia pochodzące z fragmentu bazy CIFAR-100 (25 losowych klas).

W trakcie realizacji zadania należy wykonań następujące podzadania:
1. Wygenerować dane uczące i testowe (gotowy kod).
2. Zdefiniować model głebokiej konwolucyjnej sieci neuronowej według własnego pomysłu.
3. Wytrenować zdefiniowaną sieć na danych uczących z wyodrębnieniem zbioru walidacyjnego. W procesie uczenia wykorzystać powielanie danych uczących (data augmentation, https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)
4. Ocenić skuteczność modelu na danych testowych.
5. Powtarzać etapy 2-4 aż do osiągnięcia zadowalających wyników.
6. Przeanalizować działanie stworzonej sieci, zestawienie jakie błedy w jakich klasach, zwizualizować wytrenowane filtry i mapy cech.


### Baza CIFAR-100
- Każdemu przydzielony zostaje fragment bazy CIFAR-100 zawierający dane uczące i testowe obrazów pochodzących z 25 klas.
-Klasy wybierane są losowane w zależności od `nr_albumu`.

In [ ]:
# podaj swoj nr albumu
nr_albumu = 111111

np.random.seed(nr_albumu)
(train_images_all, train_labels_all), (test_images_all, test_labels_all) = tf.keras.datasets.cifar100.load_data()
my_classes = np.random.choice(range(100), size=25, replace=False)
np.random.seed()

mask = train_labels_all[:,0]==my_classes[0]
mask2 = test_labels_all[:,0]==my_classes[0]
train_images = train_images_all[mask]
train_labels = np.ones(shape=(mask.sum(),1))*0
test_images = test_images_all[mask2]
test_labels = np.ones(shape=(mask2.sum(),1))*0
for i, cl in enumerate(my_classes[1:]):
  mask = train_labels_all[:,0]==cl
  train_images = np.concatenate((train_images, train_images_all[mask]))
  train_labels = np.concatenate((train_labels, np.ones(shape=(mask.sum(),1))*(i+1)))
  mask2 = test_labels_all[:,0]==cl
  test_images = np.concatenate((test_images, test_images_all[mask2]))
  test_labels = np.concatenate((test_labels, np.ones(shape=(mask2.sum(),1))*(i+1)))

print('Wylosowane klasy:')
print(my_classes)
print('Odpowiadające im nowe etykiety:')
print(np.array(range(25)))

# mieszanie danych
indx = np.arange(0, train_images.shape[0], 1, dtype=np.int32)
np.random.shuffle(indx)
train_images = train_images[indx]
train_labels = train_labels[indx]
indx = np.arange(0, test_images.shape[0], 1, dtype=np.int32)
np.random.shuffle(indx)
test_images = test_images[indx]
test_labels = test_labels[indx]

# normalizacja
train_images = train_images/255.
train_labels = np.int64(train_labels)
test_images = test_images/255.
test_labels = np.int64(test_labels)
print('Wymiary danych:')
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)